# ML in Cybersecurity: Task 3

## Team
  * **Team name**:  *fill this in*
  * **Members**:  *fill this in. format: name1 (email1), name2 (email2), name3 (email3),*


## Logistics
  * **Due date**: 9th December 2021, 23:59:59
  * Email the completed notebook to: `mlcysec_ws2022_staff@lists.cispa.saarland`
  * Complete this in **teams of 3**
  * Feel free to use the forum to discuss.
  
## Timeline
  * 26-Nov-2021: hand-out
  * **09-Dec-2021**: Email completed notebook
  
  
## About this Project
In this project, you will explore an application of ML to a popular task in cybersecurity: malware classification.
You will be presented with precomputed behaviour analysis reports of thousands of program binaries, many of which are malwares.
Your goal is to train a malware detector using this behavioural reports.


## A Note on Grading
The grading for this project will depend on:
 1. Vectorizing Inputs
   * Obtaining a reasonable vectorized representations of the input data (a file containing a sequence of system calls)
   * Understanding the influence these representations have on your model
 1. Classification Model  
   * Following a clear ML pipeline
   * Obtaining reasonable performances (>60\%) on held-out test set
   * Choice of evaluation metric
   * Visualizing loss/accuracy curves
 1. Analysis
   * Which methods (input representations/ML models) work better than the rest and why?
   * Which hyper-parameters and design-choices were important in each of your methods?
   * Quantifying influence of these hyper-parameters on loss and/or validation accuracies
   * Trade-offs between methods, hyper-parameters, design-choices
   * Anything else you find interesting (this part is open-ended)


## Grading Details
 * 40 points: Vectorizing input data (each input = behaviour analysis file in our case)
 * 40 points: Training a classification model
 * 15 points: Analysis/Discussion
 * 5 points: Clean code
 
## Filling-in the Notebook
You'll be submitting this very notebook that is filled-in with your code and analysis. Make sure you submit one that has been previously executed in-order. (So that results/graphs are already visible upon opening it). 

The notebook you submit **should compile** (or should be self-contained and sufficiently commented). Check tutorial 1 on how to set up the Python3 environment.


**The notebook is your project report. So, to make the report readable, omit code for techniques/models/things that did not work. You can use the final summary to provide a report about these.**

It is extremely important that you **do not** re-order the existing sections. Apart from that, the code blocks that you need to fill-in are given by:
```
#
#
# ------- Your Code -------
#
#
```
Feel free to break this into multiple-cells. It's even better if you interleave explanations and code-blocks so that the entire notebook forms a readable "story".


## Code of Honor
We encourage discussing ideas and concepts with other students to help you learn and better understand the course content. However, the work you submit and present **must be original** and demonstrate your effort in solving the presented problems. **We will not tolerate** blatantly using existing solutions (such as from the internet), improper collaboration (e.g., sharing code or experimental data between groups) and plagiarism. If the honor code is not met, no points will be awarded.

 
 ## Versions
  * v1.1: Updated deadline
  * v1.0: Initial notebook
  
  ---

In [1]:
import time 
 
import numpy as np 
import matplotlib.pyplot as plt 

import json 
import time 
import pickle 
import sys 
import csv 
import os 
import os.path as osp 
import shutil 
import pathlib
from pathlib import Path

from IPython.display import display, HTML
 
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots 
plt.rcParams['image.interpolation'] = 'nearest' 
plt.rcParams['image.cmap'] = 'gray' 
 
# for auto-reloading external modules 
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython 
%load_ext autoreload
%autoreload 2

In [2]:
# Some suggestions of our libraries that might be helpful for this project
from collections import Counter          # an even easier way to count
from multiprocessing import Pool         # for multiprocessing
from tqdm import tqdm                    # fancy progress bars

# Load other libraries here.
# Keep it minimal! We should be easily able to reproduce your code.

# We preload pytorch as an example
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [3]:
# helpers

def loading(part, whole):
    assert(isinstance(part, int))
    assert(isinstance(whole, int))
    partial = (part + 1) * 100 / whole
    sys.stdout.write('\r{}%\r'.format(int(partial * 100) / 100))
    sys.stdout.flush()
    
malware_map = {
    'NothingFound': 0,
    'Basun': 1,
    'Patched': 2,
    'Basun': 3,
    'Texel': 4,
    'Swizzor': 5,
    'Virut': 6,
    'VB': 7,
    'AutoIt': 8,
    'Agent': 9,
    'Allaple': 10
}
    
def int_to_class(i):    
    for key in malware_map.keys():
        if malware_map[key] == i:
            return key
    
def class_to_int(c):
    return malware_map[c]

# Setup

  * Download the datasets: [train](https://nextcloud.mpi-klsb.mpg.de/index.php/s/pJrRGzm2So2PMZm) (128M) and [test](https://nextcloud.mpi-klsb.mpg.de/index.php/s/zN3yeWzQB3i5WqE) (92M)
  * Unpack them under `./data/train` and `./data/test`
  * Hint: you can execute shell scripts from notebooks using the `!` prefix, e.g., `! wget <url>`

In [4]:
# Check that you are prepared with the data
! printf '# train examples (Should be 13682) : '; ls data/train | wc -l
! printf '# test  examples (Should be 10000) : '; ls data/test | wc -l

# train examples (Should be 13682) :    13682
# test  examples (Should be 10000) :    10000


Now that you're set, let's briefly look at the data you have been handed.
Each file encodes the behavior report of a program (potentially a malware), using an encoding scheme called "The Malware Instruction Set" (MIST for short).
At this point, we highly recommend you briefly read-up Sec. 2 of the [MIST](http://www.mlsec.org/malheur/docs/mist-tr.pdf) documentation.

You will find each file named as `filename.<malwarename>`:
```
» ls data/train | head
00005ecc06ae3e489042e979717bb1455f17ac9d.NothingFound
0008e3d188483aeae0de62d8d3a1479bd63ed8c9.Basun
000d2eea77ee037b7ef99586eb2f1433991baca9.Patched
000d996fa8f3c83c1c5568687bb3883a543ec874.Basun
0010f78d3ffee61101068a0722e09a98959a5f2c.Basun
0013cd0a8febd88bfc4333e20486bd1a9816fcbf.Basun
0014aca72eb88a7f20fce5a4e000c1f7fff4958a.Texel
001ffc75f24a0ae63a7033a01b8152ba371f6154.Texel
0022d6ba67d556b931e3ab26abcd7490393703c4.Basun
0028c307a125cf0fdc97d7a1ffce118c6e560a70.Swizzor
...
```
and within each file, you will see a sequence of individual systems calls monitored duing the run-time of the binary - a malware named 'Basun' in the case:
```
» head data/train/000d996fa8f3c83c1c5568687bb3883a543ec874.Basun
# process 000006c8 0000066a 022c82f4 00000000 thread 0001 #
02 01 | 000006c8 0000066a 00015000
02 02 | 00006b2c 047c8042 000b9000
02 02 | 00006b2c 047c8042 00108000
02 02 | 00006b2c 047c8042 00153000
02 02 | 00006b2c 047c8042 00091000
02 02 | 00006b2c 047c8042 00049000
02 02 | 00006b2c 047c8042 000aa000
02 02 | 00006b2c 047c8042 00092000
02 02 | 00006b2c 047c8042 00011000
...
```
(**Note**: Please ignore the first line that begins with `# process ...`.)

Your task in this project is to train a malware detector, which given the sequence of system calls (in the MIST-formatted file like above), predicts one of 10 classes: `{ Agent, Allaple, AutoIt, Basun, NothingFound, Patched, Swizzor, Texel, VB, Virut }`, where `NothingFound` roughly represents no malware is present.
In terms of machine learning terminology, your malware detector $F: X \rightarrow Y$ should learn a mapping from the MIST-encoded behaviour report (the input $x \in X$) to the malware class $y \in Y$.

Consequently, you will primarily tackle two challenges in this project:
  1. "Vectorizing" the input data i.e., representing each input (file) as a tensor
  1. Training an ML model
  

### Some tips:
  * Begin with an extremely simple representation/ML model and get above chance-level classification performance
  * Choose your evaluation metric wisely
  * Save intermediate computations (e.g., a token to index mapping). This will avoid you parsing the entire dataset for every experiment
  * Try using `multiprocessing.Pool` to parallelize your `for` loops

---

# 1. Vectorize Data

## 1.a. Load Raw Data

In [37]:
def load_content(filepath):
    '''Given a filepath, returns (content, classname), where content = [list of lines in file]'''
    with open(filepath) as file:
        lines = list(map(lambda line: line[:-2], file.readlines()))
    return lines, filepath.split(".")[2]


def load_data(data_path, nworkers=10):
    '''Returns each data sample as a tuple (x, y), x = sequence of strings (i.e., syscalls), y = malware program class'''
    raw_data_samples = []
    files = os.listdir(data_path)
    for idx, file in enumerate(files):
        loading(idx, len(files))
        lines, label = load_content(data_path + "/" + file)
        raw_data_samples.append((lines, label))
    return raw_data_samples

In [6]:
print('=> Loading training data ... ')
train_raw_samples = load_data('./data/train')

=> Loading training data ... 


In [38]:
project_mode = 'trainval'    # trainval, traintest, debug
np.random.seed(123)          # To perform the same split across multiple runs

train = list()
val = list()
test = list()

if project_mode == 'trainval':
    l = len(train_raw_samples)
    n = int(round(l * 0.8))
    train = train_raw_samples[0:n]
    val = train_raw_samples[n:l]
    
elif project_mode == 'test':
    print('=> Loading testing data ... ')
    test = load_data('./data/test')
    
elif project_mode == 'debug':
    l = len(train_raw_samples)
    n = int(round(l * 0.1))
    train = train_raw_samples[0:n]
    val = train_raw_samples[n+1:2*n+1]
else:
    raise ValueError('Unrecognized mode')
    
print('=> # Train samples = ', len(train))
print('=> # Val samples = ', len(val))
print('=> # Test  samples = ', len(test))

=> # Train samples =  10946
=> # Val samples =  2736
=> # Test  samples =  0


## 1.b. Vectorize: Setup

Make one pass over the inputs to identify relevant features/tokens.

Suggestion:
  - identify tokens (e.g., unigrams, bigrams)
  - create a token -> index (int) mapping. Note that you might have a >10K unique tokens. So, you will have to choose a suitable "vocabulary" size.

In [39]:

import copy

def get_key_idx_map(input_sets, vocab_size, n=1):
    # counter is a mapping: token -> count
    # build vectorizer using vocab_size most common elements
    key_to_idx, idx_to_key = dict(), dict()
    for idx, input_set in enumerate(input_sets):
        loading(idx, len(input_sets))
        key_to_idx = aggregate_dicts(key_to_idx, ngrams(input_set, n))
    key_to_idx = cutoff_and_rank(key_to_idx, vocab_size)
    idx_to_key = invert_dict(key_to_idx)
    return key_to_idx, idx_to_key

def ngrams(input_set, n):
    new_counter = dict()
    for idx in range(len(input_set)):
        seq = ngram(input_set, idx, n)
        if seq in new_counter:
            new_counter[seq] += 1
        elif len(seq) == n:
            new_counter[seq] = 1
    return new_counter

def ngram(input_set, idx, n):
    return tuple(input_set[idx:idx + n])

def aggregate_dicts(d1, d2):
    idx = 0
    new_dict = copy.deepcopy(d1)
    for val in d2:
        count = d2[val]
        if val in new_dict:
            new_dict[val] += count
        else:
            new_dict[val] = count
    return new_dict

def cutoff_and_rank(counter, vocab_size):
    new_counter = dict()
    counter = copy.deepcopy(counter)
    rank = 1
    vocab_size = min(vocab_size, len(counter.items()))
    for _ in range(vocab_size):
        max_value = max(counter, key=counter.get)
        new_counter[max_value] = rank
        counter[max_value] = 0
        rank += 1
    return new_counter

def invert_dict(d):
    return {value: key for key, value in d.items()}

In [40]:
MAX_VOCAB_SIZE = 50

In [41]:
uni_grams = get_key_idx_map(list(map(lambda s: s[0], train)), MAX_VOCAB_SIZE)

In [44]:
path = 'application_vocab_{}.pkl'.format(MAX_VOCAB_SIZE)
with open(path, 'wb') as file:
    d = {'token_to_idx': uni_grams[0],
          'idx_to_token': uni_grams[1]}
    pickle.dump(d, file)

## 1.c. Vectorize Data

Use the (token $\rightarrow$ index) mapping you created before to vectorize your data

In [45]:
with open(path, 'rb') as file:
    d = pickle.load(file)
    token_to_idx = d['token_to_idx']
    idx_to_token = d['idx_to_token']

In [46]:
def vectorize_raw_samples(raw_samples, nworkers=10):
    vectorized_samples = []
    for idx, sample in enumerate(raw_samples):
        loading(idx, len(raw_samples))
        lines, lable = sample
        vectorized_samples.append((vectorize(lines), lable))
        idx += 1
    return vectorized_samples

def vectorize(sample):
    counter = list()
    for idx in range(len(uni_grams[0])):
        elem = uni_grams[1][idx + 1]
        count = occurence(sample, elem)
        counter.append(count)
    return counter

def occurence(lst, obj):
    count = 0
    for idx in range(len(lst)):
        seq = ngram(lst, idx, len(obj))
        if seq == obj:
            count += 1
    return count

In [47]:
print('=> Processing: Train')
train_data = vectorize_raw_samples(train)

=> Processing: Train


In [ ]:
print('=> Processing: Val')
val_data = vectorize_raw_samples(val)

=> Processing: Val


In [ ]:
print('=> Processing: Test')
test_data = vectorize_raw_samples(test)

=> Processing: Test


In [230]:
train_x, train_y, val_x, val_y, test_x, test_y = list(), list(), list(), list(), list(), list()
    
if project_mode == 'trainval' or project_mode == 'debug':
    for vector, label in train_data:
        train_x.append(vector)
        train_y.append(class_to_int(label))
    for vector, label in val_data:
        val_x.append(vector)
        val_y.append(class_to_int(label))
else:
    for vector, label in test_data:
        test_x.append(vector)
        test_y.append(class_to_int(label))

In [231]:
assert(len(train_x) == len(train_y))
assert(len(val_x) == len(val_y))
assert(len(test_x) == len(test_y))

if project_mode == 'trainval' or project_mode == 'debug':
    trainset = TensorDataset(torch.tensor(train_x).float(), torch.tensor(train_y))
    valset = TensorDataset(torch.tensor(val_x).float(), torch.tensor(val_y))
    torch.save(trainset, 'trainset.pt')
    torch.save(valset, 'valset.pt')
else:
    testset = TensorDataset(torch.tensor(test_x).float(), torch.tensor(test_y))
    torch.save(testset, 'testset.pt')

# 2. Train Model

You will now train an ML model on the vectorized datasets you created previously.

_Note_: Although we often refer to each input as a 'vector' for simplicity, each of your inputs can also be higher dimensional tensors.

## 2.a. Helpers

In [ ]:
# Feel free to edit anything in this block

def evaluate_preds(y_gt, y_pred):
    pass


def another_helper(question):
    return 42


def save_model(model, out_path):
    pass


def save_data(eval_data, out_path):
    with open(out_path, 'wb') as wf:
        pickle.dump(eval_data, out_path)

## 2.b. Define Model

Describe your model here.

In [368]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Net, self).__init__()
        # Layer definitions
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, output_dim)
        

    def forward(self, x):
        # Forward
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

## 2.c. Set Hyperparameters

In [369]:
# Hyperparameters are defined here
in_dims = trainset[0][0].shape[0]
out_dims = 11

n_epochs = 5
batch_size = 16
lr = 0.001
loss_func = nn.CrossEntropyLoss()
malwareClassifier = Net(in_dims, out_dims)

## 2.d. Train your Model

In [370]:
loss_values = []
def train_model(model, 
                train_loader,
                num_epochs,
                learning_rate,
                loss_function):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)
    running_loss = 0.0
    
    for epoch in tqdm(range(num_epochs)):
        print(f'---------- Started Epoch {epoch} ----------')
        
        for train_data in train_loader:
            
            optimizer.zero_grad()
            
            inputs, targets = train_data
            outputs = model(inputs)
            
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        loss_value = running_loss/len(train_loader)
        loss_values.append(loss_value)
        print("Loss: {:.4f}".format(loss_value))



In [371]:
# Data Loaders
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

train_model(model=malwareClassifier, train_loader=trainloader, num_epochs=n_epochs,
            loss_function=loss_func, learning_rate=lr)


  0%|          | 0/5 [00:00<?, ?it/s]<ipython-input-368-e8a64d312a4c>:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


---------- Started Epoch 0 ----------


 20%|██        | 1/5 [00:00<00:01,  2.42it/s]

Loss: 3.3225
---------- Started Epoch 1 ----------


 40%|████      | 2/5 [00:00<00:01,  2.64it/s]

...

Loss: 7.7422
---------- Started Epoch 3 ----------


 80%|████████  | 4/5 [00:01<00:00,  2.66it/s]

Loss: 9.8817
---------- Started Epoch 4 ----------


100%|██████████| 5/5 [00:01<00:00,  2.63it/s]

Loss: 11.9802


## 2.e. Evaluate model

In [384]:
test_accuracy = []
def test(model):
    model.eval()
    correct = 0
    total = len(valset)
    with torch.no_grad():
        for val_data in valset:
            inputs, labels = val_data
            outputs = model(inputs)
            predicted = torch.max(outputs.data)
            correct += (predicted == labels).sum().item() 
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy}")
    test_accuracy.append(accuracy)

In [385]:
test(malwareClassifier)

<ipython-input-368-e8a64d312a4c>:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Test Accuracy: 0.0


## 2.f. Save Model + Data

In [387]:
# save the model 
torch.save(malwareClassifier.state_dict(), "./malwareClassifier_parameters.pt")


---

# 3. Analysis

## 3.a. Summary: Main Results

Summarize your approach and results here

## 3.b. Discussion

Enter your final summary here.

For instance, you can address:
- What was the performance you obtained with the simplest approach?
- Which vectorized input representations helped more than the others?
- Which malwares are difficult to detect and why?
- Which approach do you recommend to perform malware classification?